In [6]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('./Data/train.csv')
test_data = pd.read_csv('./Data/test.csv')
Y_test = pd.read_csv('./Data/sample_submission.csv')
pd.set_option('display.max_rows', None, 'display.max_columns', None)

train_data.shape

(1460, 81)

In [7]:
Id = test_data['Id']
Y_train = train_data.loc[:,['SalePrice']]
X_train = train_data.loc[:,:'SaleCondition']
X_test = test_data
Y_test = Y_test['SalePrice']
columns = X_train.columns

### Finding  columns that need to encode

In [8]:
temp = X_train.dtypes
labelEncodingColumns = []
for col,val in enumerate(temp):
    if val=='object':
        labelEncodingColumns.append(X_train.columns[col])
    

### Counting Nan values of each column and delete which has more than 1100 Nan values

In [9]:
for i in columns:
    print(i,":")
    print(X_train[i].isna().sum())
    print('#####################################')

Id :
0
#####################################
MSSubClass :
0
#####################################
MSZoning :
0
#####################################
LotFrontage :
259
#####################################
LotArea :
0
#####################################
Street :
0
#####################################
Alley :
1369
#####################################
LotShape :
0
#####################################
LandContour :
0
#####################################
Utilities :
0
#####################################
LotConfig :
0
#####################################
LandSlope :
0
#####################################
Neighborhood :
0
#####################################
Condition1 :
0
#####################################
Condition2 :
0
#####################################
BldgType :
0
#####################################
HouseStyle :
0
#####################################
OverallQual :
0
#####################################
OverallCond :
0
#####################################
YearBuilt :

In [10]:
for i in columns:
    if train_data[i].isna().sum() > 1100 :
       X_train = X_train.drop([i],axis=1)
       X_test = X_test.drop([i],axis=1)
       if  i in labelEncodingColumns:
            labelEncodingColumns.remove(i)
       if i in columns:
            columns = np.delete(columns,np.where(columns==i))
           

### Convert Nan values to string

In [11]:
X_train[labelEncodingColumns] = X_train[labelEncodingColumns].replace(np.nan,'fake')
X_test[labelEncodingColumns] = X_test[labelEncodingColumns].replace(np.nan,'fake')

###  Testing "fake" data type

In [12]:
for i in labelEncodingColumns:
    print(i,":")
    print(X_train[i].value_counts())
    print('#####################################')

MSZoning :
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64
#####################################
Street :
Pave    1454
Grvl       6
Name: Street, dtype: int64
#####################################
LotShape :
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64
#####################################
LandContour :
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64
#####################################
Utilities :
AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64
#####################################
LotConfig :
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: LotConfig, dtype: int64
#####################################
LandSlope :
Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: int64
#####################################
Neighborhood :
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     

### Labelling string data

In [13]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

for i in labelEncodingColumns:
    X_train[i] = labelEncoder.fit_transform(X_train[i])
    X_test[i] = labelEncoder.fit_transform(X_test[i])
    

### Filling Nan values by using knn imputer

In [14]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=10)
X_train = knn_imputer.fit_transform(X_train)
X_test = knn_imputer.fit_transform(X_test)

In [15]:
temp = pd.DataFrame(X_train,columns=columns)
X_train = temp

temp = pd.DataFrame(X_test,columns=columns)
X_test = temp

del temp

In [16]:
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(n_estimators=350)
model.fit(X_train,Y_train)
print(model.score(X_train,Y_train))



C:\Users\asus\.conda\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.9822744048185936


In [17]:
Y_predict=model.predict(X_test)